# Tier 2: Monte Carlo Convergence

**Learning Objectives:**
- Understand MC convergence properties
- Learn appropriate path counts
- Apply variance reduction techniques

**Key Insight [T1]:**
Standard error decreases as 1/√n — need 100x more paths for 10x less error.

**Duration:** ~20 minutes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from annuity_pricing.options.pricing.black_scholes import black_scholes_call

## 1. MC Price vs Number of Paths

In [ ]:
def mc_call(S, K, r, q, sigma, T, n_paths, seed=None):
    """Simple MC call pricer."""
    rng = np.random.default_rng(seed)
    drift = r - q
    Z = rng.standard_normal(n_paths)
    S_T = S * np.exp((drift - 0.5*sigma**2)*T + sigma*np.sqrt(T)*Z)
    payoffs = np.maximum(S_T - K, 0)
    price = np.exp(-r*T) * payoffs.mean()
    se = np.exp(-r*T) * payoffs.std() / np.sqrt(n_paths)
    return price, se

# Parameters
S, K, r, q, sigma, T = 100, 100, 0.05, 0.02, 0.20, 1.0
bs_price = black_scholes_call(S, K, r, q, sigma, T)

# Convergence study
path_counts = [100, 500, 1000, 5000, 10000, 50000, 100000]
mc_prices = []
mc_errors = []

for n in path_counts:
    price, se = mc_call(S, K, r, q, sigma, T, n, seed=42)
    mc_prices.append(price)
    mc_errors.append(se)

# Plot
plt.figure(figsize=(10, 5))
plt.errorbar(path_counts, mc_prices, yerr=[2*e for e in mc_errors], 
             fmt='o-', capsize=5, label='MC Price ± 2 SE')
plt.axhline(y=bs_price, color='red', linestyle='--', label=f'BS Price (${bs_price:.4f})')
plt.xscale('log')
plt.xlabel('Number of Paths')
plt.ylabel('Call Price ($)')
plt.title('Monte Carlo Convergence to Black-Scholes')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\nConvergence Results:")
print(f"{'Paths':>10} {'MC Price':>12} {'SE':>10} {'Error':>10}")
print("-" * 45)
for n, p, e in zip(path_counts, mc_prices, mc_errors):
    print(f"{n:>10,} ${p:>11.4f} ${e:>9.4f} {abs(p-bs_price)/bs_price:>9.2%}")

## 2. Standard Error Scaling

SE ∝ 1/√n

- 100 paths: SE ~ 0.30
- 10,000 paths: SE ~ 0.03 (10x fewer paths, 10x smaller SE)
- 1,000,000 paths: SE ~ 0.003

## 3. Practical Guidelines [T3]

| Use Case | Paths | Error Target |
|----------|-------|-------------|
| Quick estimate | 10,000 | ~1-2% |
| Production pricing | 100,000 | ~0.5% |
| Validation/research | 1,000,000+ | ~0.1% |

**Rule of thumb**: For ~1% error, use ~100,000 paths.

## 4. Key Takeaways

1. **MC converges to BS** for vanilla options
2. **100k paths** is standard for production
3. **SE ~ 1/√n** — need 100x paths for 10x accuracy
4. **Verify against BS** before using for path-dependent products